In [11]:
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain_core.tools import StructuredTool
from langchain_community.utilities.serpapi import SerpAPIWrapper
import os

In [12]:
# Set the envionrment variable for the API key:
os.environ['SERPAPI_API_KEY'] = 'INSERT_API_KEY_HERE'

In [13]:
# Define Tools:
def google_search(query: str):
    """Tool to search Google"""
    return SerpAPIWrapper().run(query)

def save_interview(raw_interview_text: str):
    """Tool to save the interview. You must pass the entire interview and conversation in here.
    The interview will then be saved to a local file. Remember to include all of the previous chat messages.
    Include all of the messages with the user and the AI, here is a good response:
    AI: some text
    Human: some text
    ...
    ---
    """
    # Save to local file:
    with open("interview.txt", "w") as f:
        f.write(raw_interview_text)
    return f"Interview saved! Content: {raw_interview_text}. File: interview.txt. You must tell the user that the interview is saved."

def get_pokemon(pokemon_name: str):
    """Tool to get a single Pokemon"""
    import requests
    clean_pokemon_name = pokemon_name.lower()
    # Get the Pokemon
    try:
        pokemon = requests.get(f"https://pokeapi.co/api/v2/pokemon/{clean_pokemon_name}").json()
        if pokemon:
            return f"Pokemon found! Name: {pokemon['name']}. Height: {pokemon['height']}. Weight: {pokemon['weight']}."
        else:
            return "Pokemon not found!"
    except Exception as e:
        print(e)
        return "Pokemon not found! Try a different pokemon."

In [14]:
# Convert functions to Structured Tools:
google_search = StructuredTool.from_function(google_search)
save_interview = StructuredTool.from_function(save_interview)
get_single_pokemon = StructuredTool.from_function(get_pokemon)

In [15]:
from pydantic.v1 import BaseModel
from typing import Optional, Type, Union, Literal, List
from langchain_core.tools import BaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.tools.base import ToolException
from langchain_core.documents import Document
import tempfile
import requests

class ArgumentType(BaseModel):
    url: str
    file_type: Union[Literal["pdf"], Literal["txt"]]

class SummarizeFileFromURL(BaseTool):
    name = "SummarizeFileFromURL"
    description = "Summarize a file from a URL."
    args_schema: Type[ArgumentType] = ArgumentType

    def _run(
        self,
        url: str,
        file_type: Union[Literal["pdf"], Literal["txt"]],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        from langchain_community.document_loaders import PyPDFLoader, TextLoader
        from langchain.chains.summarize import load_summarize_chain

        response = requests.get(url)
        if response.status_code != 200:
            raise ToolException("The URL is not valid. Please try another URL.")

        if file_type == "pdf":
            with tempfile.NamedTemporaryFile(suffix=".pdf", delete=True) as tmp:
                tmp.write(response.content)
                pdf_loader = PyPDFLoader(file_path=tmp.name)
                documents: List[Document] = pdf_loader.load()
                
        else:
            with tempfile.NamedTemporaryFile(suffix=".txt", delete=True, mode="w+t") as tmp:
                tmp.write(response.text)
                text_loader = TextLoader(tmp.name)
                documents: List[Document] = text_loader.load()

        chain = load_summarize_chain(llm=ChatOpenAI(), chain_type="map_reduce")
        return chain.run(documents)

    async def _arun(
        self,
        url: str,
        file_type: Union[Literal["pdf"], Literal["txt"]],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This tool does not support async yet!")
    

def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

file_summarizer = SummarizeFileFromURL()
file_summarizer.handle_tool_error = _handle_error

In [16]:
# Tools:
tools = [google_search, save_interview, get_single_pokemon, file_summarizer]

In [17]:
# Create the llm:
llm = ChatOpenAI(temperature=0)

# Create the ChatPromptTemplate:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the system message:
system_message = SystemMessage(
    content="""Act as a helpful AI interview assistant, 
               Always ask a follow-up question, especially after returning results.
               Ask one question at a time.
               Always be in interview mode or tool mode.
               Ask the user about the desired topic.
                               """
)

# Initialize memory:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# Create the Prompt:
prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(
            variable_name="agent_scratchpad"
        ),  # This is where the agent will write/read its messages from
        MessagesPlaceholder(variable_name="history"), # This is where the user will write/read their messages from as memory
        ("user", "{input}"),
    ]
)

agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [18]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

# Formats the python function tools into JSON schema and binds them to the model:
llm_with_tools = llm.bind(
    functions=[convert_to_openai_function(t) for t in tools]
)

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

# Setting up the agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "history": RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [19]:
# Create an agent executor by passing in the agent, tools and memory:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, max_iterations=30)

In [20]:
agent_executor.invoke({"input": "My name is James, I'm a computer programmer and I like to play cosy games."})



> Entering new AgentExecutor chain...
Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?

> Finished chain.


{'input': "My name is James, I'm a computer programmer and I like to play cosy games.",
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games."),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?")],
 'output': "Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?"}

In [21]:
agent_executor.invoke({"input": "What is my name and my profession?"})



> Entering new AgentExecutor chain...
Your name is James, and you mentioned that you are a computer programmer. It's great to know more about you! Is there anything specific you would like to know or talk about related to computer programming or cosy games?

> Finished chain.


{'input': 'What is my name and my profession?',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games."),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?"),
  HumanMessage(content='What is my name and my profession?'),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's great to know more about you! Is there anything specific you would like to know or talk about related to computer programming or cosy games?")],
 'output': "Your name is James, and you mentioned that you are a computer programmer. It's great to know more about you! Is there anything specific you would like to know or talk about related to computer programming or cosy games?"}

In [22]:
agent_executor.invoke({"input": "Get information about Pikachu"})



> Entering new AgentExecutor chain...

Invoking: `get_pokemon` with `{'pokemon_name': 'Pikachu'}`


Pokemon found! Name: pikachu. Height: 4. Weight: 60.
Invoking: `get_pokemon` with `{'pokemon_name': 'Pikachu'}`


Pokemon found! Name: pikachu. Height: 4. Weight: 60.Pikachu is a popular Pokémon known for its electric abilities. It has a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu or any other Pokémon?

> Finished chain.


{'input': 'Get information about Pikachu',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games."),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?"),
  HumanMessage(content='What is my name and my profession?'),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's great to know more about you! Is there anything specific you would like to know or talk about related to computer programming or cosy games?"),
  HumanMessage(content='Get information about Pikachu'),
  AIMessage(content='Pikachu is a popular Pokémon known for its electric abilities. It has a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu or any other Pokémon?')],
 'output': 'Pikachu is a popular Pokémon known for its electric abilities. I

In [23]:
agent_executor.invoke({"input": "Get information about Charmander"})



> Entering new AgentExecutor chain...

Invoking: `get_pokemon` with `{'pokemon_name': 'Charmander'}`


Pokemon found! Name: charmander. Height: 6. Weight: 85.Charmander is a Fire-type Pokémon with a height of 6 and a weight of 85. It's a classic and beloved Pokémon known for its fiery tail. Is there anything else you would like to know about Charmander or any other Pokémon?

> Finished chain.


{'input': 'Get information about Charmander',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games."),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?"),
  HumanMessage(content='What is my name and my profession?'),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's great to know more about you! Is there anything specific you would like to know or talk about related to computer programming or cosy games?"),
  HumanMessage(content='Get information about Pikachu'),
  AIMessage(content='Pikachu is a popular Pokémon known for its electric abilities. It has a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu or any other Pokémon?'),
  HumanMessage(content='Get information about Charmander'),
  AIMessage(co

In [24]:
agent_executor.invoke(
    {
        "input": "I want a comprehensive summary of our conversation! Please can you save it?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `save_interview` with `{'raw_interview_text': "User: My name is James, I'm a computer programmer and I like to play cosy games.\nAssistant: Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?\nUser: What is my name and my profession?\nAssistant: Your name is James, and you mentioned that you are a computer programmer. It's great to know more about you! Is there anything specific you would like to know or talk about related to computer programming or cosy games?\nUser: Get information about Pikachu\nAssistant: Pikachu is a popular Pokémon known for its electric abilities. It has a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu or any other Pokémon?\nUser: Get information about Charmander\nAssistant: Charmander is a Fire-type Pokémon with a height of 6 and a weight of 85. It's 

{'input': 'I want a comprehensive summary of our conversation! Please can you save it?',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games."),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific topic or question you'd like to discuss or learn more about today?"),
  HumanMessage(content='What is my name and my profession?'),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's great to know more about you! Is there anything specific you would like to know or talk about related to computer programming or cosy games?"),
  HumanMessage(content='Get information about Pikachu'),
  AIMessage(content='Pikachu is a popular Pokémon known for its electric abilities. It has a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu or any other Pokémon?'),
  HumanMessage(content='Get inf

In [25]:
# This will fail because the agent is not able to summarize the file type (docx) yet: 
try:
    agent_executor.invoke({"input":"I want to summarize this https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202022.docx"})
except Exception as e:
    error_message = str(e)
    chat = ChatOpenAI()
    clean_error_message = chat.invoke([SystemMessage(content=error_message + 'Clean the error message, this will be shown to a user.')])
    print(clean_error_message.content)



> Entering new AgentExecutor chain...

Invoking: `SummarizeFileFromURL` with `{'url': 'https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202022.docx', 'file_type': 'pdf'}`




invalid pdf header: b'PK\x03\x04\x14'
EOF marker not found


The stream has ended unexpectedly. Please try refreshing the page or checking your internet connection.


In [26]:
# This line will take several minutes as we are summarizing a 41 page .pdf file:
agent_executor.invoke(
    {
        "input": "I want to summarize this https://storage.googleapis.com/oreilly-content/pokemon-rules.pdf "
    }
)



> Entering new AgentExecutor chain...

Invoking: `SummarizeFileFromURL` with `{'url': 'https://storage.googleapis.com/oreilly-content/pokemon-rules.pdf', 'file_type': 'pdf'}`




/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The Pokémon Trading Card Game Rules provide guidelines for setup, gameplay mechanics, and winning conditions. Players build decks, battle opponents, and strategize to knock out their opponent's Pokémon using special abilities, energy cards, and evolution mechanics. The game involves drawing cards, playing Pokémon, using Trainer cards, and attacking to win by taking all Prize cards or knocking out the opponent's Pokémon. Special conditions like Poisoned and Paralyzed must be addressed, and powerful cards like Pokémon VMAX and TAG TEAM Pokémon-GX are introduced. Trainers aim to evolve Pokémon, build decks, and understand different card types to win the game.
Invoking: `SummarizeFileFromURL` with `{'url': 'https://storage.googleapis.com/oreilly-content/pokemon-rules.pdf', 'file_type': 'pdf'}`


The Pokémon Trading Card Game Rules outline guidelines for playing the game, including battling with Pokémon cards, evolving Pokémon, and winning by knocking out opponents' Pokémon. The rules cover

KeyboardInterrupt: 